In [ ]:
# Code Written by Eric-Finley Robins
# Q-Learning algorithm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from degree_freedom_queen import *
from degree_freedom_king1 import *
from degree_freedom_king2 import *
from generate_game import *
from Chess_env import *
 
N_games = 10000 
size_board = 4
env = Chess_Env(size_board)
S ,X, allowed_a = env.Initialise_game()
np.random.seed(3420)

input_neurons = np.shape(X)[0]    
hidden_neurons = 200                
output_neurons = np.shape(allowed_a)[0]   

w1 = np.random.randn(hidden_neurons, input_neurons) * np.sqrt(1/input_neurons)
w2 = np.random.randn(output_neurons, hidden_neurons) * np.sqrt(1/hidden_neurons)
b1 = np.zeros((hidden_neurons,))
b2 = np.zeros((output_neurons,))
 
# Hyperparameters
epsilon = 0.15     
decay = 0.99985     
discount = 0.85        
learning_rate = 0.0035        

# Finds indices of allowed actions 
# Generates random integer
# Perfoms either greedy or random selection from allowed integers
def EpsilonGreedy_Policy(output_out, epsilon, allowed_a):
    
    allowed_i = np.where(allowed_a == 1)[0]

    if np.random.uniform(0, 1) < epsilon:
        a_agent = allowed_i[np.random.randint(len(allowed_i))]
    else:
        output_values = output_out[allowed_i]
        a_agent = allowed_i[np.argmax(output_values)]
 
    return a_agent  

wins = 0
wins_ = []

move_total = 0
moves_ = []

for games in range(N_games + 1):
 
    epsilon *= decay
    Done = 0                                                            
    moves = 0
    S, X, allowed_a = env.Initialise_game()      

    # For calculating ema's
    if games % 100 == 0:
        moves_.append(move_total/100)
        move_total = 0
        wins_.append(wins)
        wins = 0

    # Loop for each game
    while Done == 0:                     
        
        # Forward Prop for action determination
        hidden_in = np.dot(w1, X) + b1
        hidden_out = (hidden_in > 0)* hidden_in
 
        output_in = np.dot(w2, hidden_out) + b2
        output = (output_in > 0) * output_in
        output_masked = output * allowed_a.flatten()
 
        agent = EpsilonGreedy_Policy(output, epsilon, allowed_a)
 
        S_next, X_next, allowed_a_next, R, Done = env.OneStep(agent)       
        moves += 1

        if R == 1:
            wins += 1

        Q_current = output[agent]
 
        filter = np.zeros((output_neurons, ))
        filter[agent] = 1
    
        if Done==1:
            
            Error = R - Q_current # Temporal Error Calculation 

            # Backpropagation of error
            reluOutput = np.where(output_in > 0, 1, 0)    
            delta2 = reluOutput * Error * filter
 
            dw2 = np.outer(delta2, hidden_out)
            db2 = delta2
 
            reluInput = np.where(hidden_in > 0, 1, 0)
            delta1 = reluInput * np.dot(w2.T, delta2)
 
            dw1 = np.outer(delta1, X)
            db1 = delta1
            move_total += moves

        else:
            
            # Forward prop for future reward prediction
            hidden_in_next = np.dot(w1, X_next) + b1
            hidden_out_next = (hidden_in_next > 0) * hidden_in_next
            output_in_next = np.dot(w2, hidden_out_next) + b2
            output_next = (output_in_next > 0) * output_in_next
            
            Q_max = np.max(output_next * allowed_a_next.flatten())      # Maximum Q-value in the next state
            Q_target = R + discount * Q_max                             # Q-target for the current state-action pair
            Error = Q_target - Q_current                                # Temporal Error 
 
            # Backpropagation
            reluOutput = np.where(output_in > 0, 1, 0)
            delta2 = reluOutput * Error * filter
 
            dw2 = np.outer(delta2, hidden_out)
            db2 = delta2
 
            # Backpropagation: hidden layer -> input layer
            reluInput = np.where(hidden_in > 0, 1, 0)
            delta1 = reluInput * np.dot(w2.T, delta2)
 
            dw1 = np.outer(delta1, X)
            db1 = delta1

        # Update Rules
        w1 += learning_rate * dw1
        w2 += learning_rate * dw2
        b1 += learning_rate * db1
        b2 += learning_rate * db2            

        # Passing forward from the step taken
        allowed_a = allowed_a_next
        S = S_next
        X = X_next

# X Axis labels
x_ticks = np.linspace(0, N_games, num=int(N_games/100)+1)

# Plotting EMA Graph for win %
wins_ = wins_
wins_series = pd.Series(wins_)
wins_ema = wins_series.ewm(alpha=0.2).mean()

plt.plot(x_ticks, wins_ema)
plt.xlabel('Games Played')
plt.xticks(x_ticks[::20])
plt.ylabel('Reward Earned')
plt.title('Q-learning Average Reward Earned')
plt.show()

# Plotting EMA Graph for avg. moves taken
moves_ = moves_
moves_series = pd.Series(moves_)
moves_ema = moves_series.ewm(alpha=0.2).mean()

plt.plot(x_ticks[1:], moves_ema[1:])
plt.xlabel('Games Played')
plt.xticks(x_ticks[::20])
plt.ylabel('Moves Taken')
plt.title('Q-Learning Average Moves Taken')
plt.show()
